# 構造用合板
# （structural plywood）
参考：熱湿気計算と物性値WG用  
[1]  齋藤宏昭、土屋喬雄：冷房時における夏型壁体内結露に関する研究、その１　人工気候室における非定常結露実験、日本建築学会計画系論文集　第519号、pp39-45、1999年5月  
[2] 土屋喬雄、黒木勝一、齋藤宏昭：冷房時の夏型壁体内結露に関する研究 （その 3） 1次元モデルの検証と壁体内結露の長期予測、日本建築学会大会学術講演梗概集 pp.269-270、1996年9月  
[3]　独立行政法人　建築研究所、カナダ天然資源省　編：第3回　日加住宅R&D専門家作業会合　「断熱外皮の設計」講演予稿集


In [1]:
module structural_plywood

# 空隙率
const psi = 0.8 # おおよその値

# 材料密度
const row = 504.0 #kg/m3

# 比熱
const C = 2220.0 #J/kg

# 水の密度
const roww = 1000.0 #kg/m3
#理想気体定数
const R = 8.314 # J/(mol K)
# 水のモル質量
const Mw = 0.018 # kg/mol
# 水蒸気のガス定数
Rv = R / Mw # J/(kg K)
# 水の熱容量
const croww = 1000.0 * 4.18605e+3

# miu ⇒　rh の変換係数
function convertMiu2RH( ;temp::Float64, miu::Float64 );
    return exp( miu / Rv / temp )
end

# 熱容量
get_crow( ;phi::Float64 ) = C * row + croww * phi
get_crow( cell ) = get_crow( phi = get_phi( cell ) )

### 水分特性 ###
# 含水率
# 関数系　phi = a * ( b / ( rh + c ) + d ) [kg/kg]
# 相対湿度の閾値 rh_TH
a = [ 1.0, 1.0 ]
b = [ -1.03289, -0.01227744 ]
c = [ 2.91545, -1.05979 ]
d = [ 0.35428, 0.0347136 ]
rh_TH = [ 0.669 ]

function get_phi( ;rh::Float64 )
    if rh < rh_TH[1]
        phi = ( a[1] * ( b[1] / ( rh + c[1] ) + d[1] ) )
    elseif rh >= rh_TH[1]
        phi = ( a[2] * ( b[2] / ( rh + c[2] ) + d[2] ) )
    end
    return phi
end

get_phi( cell ) = get_phi( rh = convertMiu2RH( temp = cell.temp, miu = cell.miu ) )

# 含水率の水分化学ポテンシャル微分
drh_dmiu( temp::Float64, miu::Float64 ) = - ( 1.0 / Rv / temp ) * exp( miu / Rv / temp )
# dphi/dmiu = dphi/drh * drh/dmiu
function get_dphi( ;temp::Float64, miu::Float64, rh::Float64 )
    if rh < rh_TH[1]
        dphi_drh = ( a[1] * ( b[1] / (( rh + c[1] )^2) ) )
    elseif rh >= rh_TH[1]
        dphi_drh = ( a[2] * ( b[2] / (( rh + c[2] )^2) ) )
    end
    return dphi_drh * drh_dmiu( temp, miu )
end
get_dphi( cell ) = get_dphi( temp = cell.temp, miu = cell.miu, rh = convertMiu2RH( temp = cell.temp, miu = cell.miu ) )

# 含水率から水分化学ポテンシャルの算出
function get_miu_by_phi( ;temp::Float64, phi::Float64 )
    if phi < get_phi( rh = rh_TH[1])
        rh = b[1] / (phi / a[1] - d[1]) - c[1]
    elseif phi >= get_phi( rh = rh_TH[1])
        rh = b[2] / (phi / a[2] - d[2]) - c[2]
    end
    return Rv * temp * log( rh )
end
get_miu_by_phi( cell ) = get_miu_by_phi( temp = cell.temp, phi = get_phi( cell ) )

### 移動特性 ###
# 熱伝導率
const lam = 0.1

# 湿気依存
get_lam( ;phi::Float64 ) = lam
get_lam( cell ) = get_lam( phi = get_phi( cell ) )

# 透水係数
get_dw( ;phi::Float64 ) = 0.0
get_dw( cell ) = 0.0

# 透湿率（湿気伝導率）
# kg / kg DA
dah_dpv(pv::Float64) = 0.622 / ( 101325.0 - pv ) + 0.622 * pv / ( ( 101325.0 - pv )^2 )
dah_dpv() = dah_dpv(1700.0)
get_dp() = 1.46e-7 * dah_dpv() #絶乾時の透湿率[kg/m Pa s]
get_dp( cell ) = get_dp()

end

Main.structural_plywood

In [2]:
structural_plywood.get_dphi(temp = 293.15, miu = -1000.0, rh = 0.9)

1.1633825353793242e-5